In [78]:
import json
import requests
import datetime as dt
import pickle
import pandas as pd

# An api key is emailed to you when you sign up to a plan
api_key = 'da3195c8356b3b0ee3e5a6682ed9c537'


In [79]:
from datetime import datetime, timezone
print(datetime.now())              # timezone
print(datetime.now(timezone.utc))  # coordinated universal time 

2021-01-08 08:46:19.451245
2021-01-08 14:46:19.451245+00:00


In [80]:
def get_odds_data():
    sports_response = requests.get('https://api.the-odds-api.com/v3/odds', params={
        'api_key': api_key,
        'sport': 'basketball_ncaab',
        'region': 'us',
        'oddsFormat': 'american'
    })


    sports_json = json.loads(sports_response.text)
    date = dt.datetime.today().date().strftime('%Y-%m-%d')
    with open(str(date)+'.json', 'w') as outfile:
        json.dump(sports_json, outfile)
    

In [81]:
print("https://api.the-odds-api.com/v3/odds/?apiKey=da3195c8356b3b0ee3e5a6682ed9c537&sport=basketball_ncaab&region=us&mkt=h2h&oddsFormat=american")

https://api.the-odds-api.com/v3/odds/?apiKey=da3195c8356b3b0ee3e5a6682ed9c537&sport=basketball_ncaab&region=us&mkt=h2h&oddsFormat=american


In [82]:
get_odds_data()

In [83]:
date = dt.datetime.today().date().strftime('%Y-%m-%d')
with open(str(date)+'.json') as f:
    sports_json = json.load(f)

In [84]:
def getTeamData(teamid):
    teamid = teamid.replace(" ", "%20")
    baseUrl="https://fastbreakstats-server-main-hqc7w3njda-uc.a.run.app/teams/"
    url=baseUrl+teamid

    r = requests.get(url = url).json()
    bpival = r['bpi']
    kpval = r['kp']
    srsval = r['srs']
    teamID = r['teamID']
    return kpval, bpival, srsval, teamID

In [92]:
getTeamData('134')

(23.32, 10.7, 17.54)

In [85]:
def get_needed_odds(conf):
    #This function calculates the odds needed for a bet to have an expected value of .1
    #An EV of .1 means a $.10 expected return on $1 over time
    #The normal formula is EV = (payout * confidence) - (bet * (1-conf))
    #payout is the expected profit on a bet.
    #confidence is models % chance of winning. 1-conf is for the other team
    #The bet is always going to be $1 for simple math which removes the need for the bet variable
    #below equation is rearranged to isolate and solve for the payout
    payout = round(((.1+(1-conf))/conf),4) 
    #convert payout to american odds
    if 100/payout > 100:
        odds = round(-100/payout)
    else:
        odds = round(100*payout)
    return odds

In [86]:
def get_ev(odds,conf):
    #This calculates the Expected Return on a $1 bet with the actual given bookie odds we pass in
    #EV of .1 means a 10 cent return on a $1 bet over time
    if int(odds) > 0:
        payout = odds/100
    else:
        payout = -100/odds
    ev = (conf * payout) - (1-conf)
    return ev

In [87]:
def get_kelly_bet(odds,conf):
    if int(odds) > 0:
        dec = 1+(odds/100)
    else:
        dec = 1-(100/odds)
    kb = ((dec*conf) - (1-conf))/dec
    return kb

In [88]:
def print_games():
    url="https://fastbreakstats-server-main-hqc7w3njda-uc.a.run.app/teams"
    r = requests.get(url = url).json()
    teams = pd.DataFrame(r)
#     teams = pd.read_csv('teams_table.csv')
    loaded_model = pickle.load(open('6yr_LR.sav', 'rb'))
    for game in sports_json['data']:
        #Loop through and get odds
        odds = ['n/a','n/a']
        for site in game['sites']:
            if site['site_key'] == 'bovada':
                odds = site['odds']['h2h']
                sitex = "bovada"
                break
            if site['site_key'] == 'draftkings':
                odds = site['odds']['h2h']
                sitex = "DK"
            if site['site_key'] == 'williamhill_us':
                odds = site['odds']['h2h']
                sitex = "WH"
            if site['site_key'] == 'fanduel':
                odds = site['odds']['h2h']
                sitex = "FD"

        if odds == ['n/a','n/a']:
            print("There are no odds for {0}".format(game['teams']))
            print()
            continue

        home = game['home_team']
        if game['home_team'] == game['teams'][0]:
            away = game['teams'][1]
            home_odds = odds[0]
            away_odds = odds[1]
        else:
            away = game['teams'][0]
            home_odds = odds[1]
            away_odds = odds[0]

        try:
            home = home.replace("-", " ")
            home = home.replace("(", "")
            home = home.replace(")", "")
            home = home.replace(".", "")
            away = away.replace("-", " ")
            away = away.replace("(", "")
            away = away.replace(")", "")
            away = away.replace(".", "")
            homekp, homebpi, homesrs = getTeamData(str(teams.loc[teams['odds_api_name'] == home]['teamID'].values[0]))
            awaykp, awaybpi, awaysrs = getTeamData(str(teams.loc[teams['odds_api_name'] == away]['teamID'].values[0]))
        except Exception as e:
            print(e)
            print(game['teams'])
            print()
            continue

        res = loaded_model.predict_proba([[float(awaykp), float(awaybpi), float(awaysrs), float(homekp), float(homebpi), float(homesrs)]])
    #     print(res[0])
#         print("{0} @ {1}".format(away,home))
        away_needed_odds = get_needed_odds(res[0][0])    
        home_needed_odds = get_needed_odds(res[0][1])
        away_ev = get_ev(away_odds,res[0][0])
        home_ev = get_ev(home_odds,res[0][1])
        away_kb = get_kelly_bet(away_odds,res[0][0])
        home_kb = get_kelly_bet(home_odds,res[0][1])
        if away_odds > away_needed_odds:
            print("{0} @ {1} on {2}".format(away,home,sitex))
            print("{0} needs {1} and has {2} odds with an Expected Value of {3}. Kelly Bet {4}".format(away,away_needed_odds,away_odds,away_ev,away_kb))
            print()
        if home_odds > home_needed_odds:
            print("{0} @ {1} on {2}".format(away,home,sitex))
            print("{0} needs {1} and has {2} odds with an Expected Value of {3}. Kelly Bet {4}".format(home,home_needed_odds,home_odds,home_ev,home_kb))
            print()

In [89]:
print_games()

c:\users\blake\appdata\local\programs\python\python36\lib\site-packages\sklearn\base.py:318: UserWarning: Trying to unpickle estimator LogisticRegression from version 0.22.1 when using version 0.22.2.post1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


Manhattan Jaspers @ Quinnipiac Bobcats on bovada
Quinnipiac Bobcats needs -279.0 and has -150 odds with an Expected Value of 0.3495467512771758. Kelly Bet 0.6955648812260888

Rider Broncs @ Niagara Purple Eagles on bovada
Niagara Purple Eagles needs -325.0 and has -310 odds with an Expected Value of 0.11260403073410963. Kelly Bet 0.7211970235565335

index 0 is out of bounds for axis 0 with size 0
["Mt. St. Mary's Mountaineers", 'St. Francis BKN Terriers']

Oakland Golden Grizzlies @ Green Bay Phoenix on DK
Green Bay Phoenix needs -136.0 and has 112 odds with an Expected Value of 0.34305583053926836. Kelly Bet 0.46064751497030015

Georgia Southern Eagles @ Appalachian St Mountaineers on bovada
Appalachian St Mountaineers needs -193.0 and has -145 odds with an Expected Value of 0.2251649535685719. Kelly Bet 0.5624003019459015

Ohio Bobcats @ Toledo Rockets on DK
Toledo Rockets needs -252.0 and has -175 odds with an Expected Value of 0.23711861654400002. Kelly Bet 0.6518755841697852

Purd